In [174]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.grid_search import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import sklearn.cross_validation
from sklearn.cross_validation import cross_val_score

In [186]:
def setPred(col):
    if(col > 0):
        return 1
    else:
        return 0
    
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior  
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

# preparar/tweks datos 

In [215]:
features = pd.read_csv("features0,1.csv",low_memory = False)
prediccion = pd.read_csv("csv/labels_training_set.csv")

In [216]:
#mergeo features y la info q nos tiran, lleno con 0 los q no se
train = pd.merge(prediccion,features,on="person",how= "left").dropna()

In [217]:
prediccion = train["label"]

In [218]:
test = features.loc[~features["person"].isin(train["person"])]

In [219]:
listaDeCategoricos = ["condition_no_convercion","model_no_convercion","color_no_convercion","search_engine"\
                      ,"nombre mas usado dia","ultimo_dia_de_la_semana_activo","ultimo_evento_registrado_cmode"]
for word in listaDeCategoricos:
    train[word],test[word] =  target_encode(trn_series = train[word],  \
                                            tst_series = test[word],\
                                            target=prediccion, \
                                            min_samples_leaf=50,\
                                            smoothing=5,\
                                            noise_level=0.01)


/home/tito/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [220]:
train = train.drop(["ultimo_dia_de_la_semana_activo"],axis = 1)


In [221]:
train = train.drop(["person","label"],axis = 1)

In [79]:
for word in features.columns:
    features[word] = pd.to_numeric(features[word],errors = "coerce")

In [222]:

X_train, X_test, y_train, y_test = train_test_split(train, prediccion, test_size=0.30, random_state=42)

# Random Forest
# historial cambios tweaks | score

columnas: 
'ad campaign hit', 'brand listing', 'checkout', 'generic listing',
       'lead', 'search engine hit', 'searched products', 'staticpage',
       'viewed product', 'visited site', 'level_0', 'storage_no_convercion',
       'numero mas usado dia', 'condition_no_convercion2',
       'color_no_convercion2', 'search_engine2', 'nombre mas usado dia2']

test_size = 0.3
cv = 20
n_estimator 50 | .78 (Kaggle .76)
n_estimator 100 | .79 

cambio smootingbasado en: conversiones previas -> conversiones futuras

test_size = 0.3
cv = 20
n_estimator 100 | .80 (Kaggle .78)

agrego ultimo evento

test_size = 0.3
cv = 20
n_estimator 100 | .793


agrego dias hasta el 1/6

test_size = 0.3
cv = 20
n_estimator 100 | .786

agrego: ultimo dia de semana,ultimo dia,ultimo mes,ultimo hora
saco: dias hasta el 1/6
test_size = 0.3
cv = 20
n_estimator 100 | .78 (Kaggle .778)


https://www.youtube.com/watch?v=BSUMBBFjxrY

In [155]:
randomforesttree = RandomForestRegressor(random_state = None, n_jobs=-1)
randomforesttree.get_params()

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [156]:
param_grid= {'max_features': ["auto"], 'n_estimators': [100], 'criterion': ['mse']}
grid_drop = GridSearchCV(randomforesttree, param_grid, cv=20, scoring='roc_auc',n_jobs = -1,pre_dispatch = 4)
grid_drop.fit(X_train, y_train)
print(grid_drop.best_params_, grid_drop.best_score_)

{'max_features': 'auto', 'n_estimators': 100, 'criterion': 'mse'} 0.7860090608727548


In [157]:
result = grid_drop.best_estimator_.predict(X_test)

In [158]:
roc_auc_score(y_test, result)

0.811093916147653

In [97]:
feature_importances = pd.DataFrame(grid_drop.best_estimator_.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

In [98]:
feature_importances

,importance
checkout,0.082237
nombre mas usado dia,0.076558
ultimo_evento_registrado_cmode,0.075809
color_no_convercion,0.075315
ultimo_dia_de_la_semana_activo,0.072172
search_engine,0.070993
condition_no_convercion,0.070082
model_no_convercion,0.062863
viewed product,0.048716
numero mas usado dia,0.044006


# Gradient Boost
# historial cambios tweaks | score 

columnas: 
'ad campaign hit', 'brand listing', 'checkout', 'generic listing',
       'lead', 'search engine hit', 'searched products', 'staticpage',
       'viewed product', 'visited site', 'level_0', 'storage_no_convercion',
       'numero mas usado dia', 'condition_no_convercion2',
       'color_no_convercion2', 'search_engine2', 'nombre mas usado dia2']


cambio smootingbasado en: conversiones previas -> conversiones futuras

agrego ultimo evento

agrego dias hasta el 1/6

test_size = 0.3
cv = 20
n_estimator 100 | .82 (Kaggle .815)


agrego: ultimo dia de semana

test_size = 0.3
cv = 20
n_estimator 100 | .820

agrego:ultimo dia,ultimo mes,ultimo hora

test_size = 0.3
cv = 20
n_estimator 100 | .819




In [223]:
gradientboost = GradientBoostingRegressor(random_state = None)
gradientboost.get_params()

{'alpha': 0.9,
 'init': None,
 'learning_rate': 0.1,
 'loss': 'ls',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'presort': 'auto',
 'random_state': None,
 'subsample': 1.0,
 'verbose': 0,
 'warm_start': False}

In [224]:
param_grid= {'max_features': ["auto"], 'n_estimators': [30], 'loss': ['ls']}
grid_drop = GridSearchCV(gradientboost, param_grid, cv=20, scoring='roc_auc')
grid_drop.fit(X_train, y_train)
print(grid_drop.best_params_, grid_drop.best_score_)

{'max_features': 'auto', 'loss': 'ls', 'n_estimators': 30} 0.8203484693975962


In [225]:
result2 = grid_drop.best_estimator_.predict(X_test)

In [226]:
roc_auc_score(y_test, result2)

0.8445500405083801

In [214]:
feature_importances2 = pd.DataFrame(grid_drop.best_estimator_.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
feature_importances

,importance
checkout,0.082237
nombre mas usado dia,0.076558
ultimo_evento_registrado_cmode,0.075809
color_no_convercion,0.075315
ultimo_dia_de_la_semana_activo,0.072172
search_engine,0.070993
condition_no_convercion,0.070082
model_no_convercion,0.062863
viewed product,0.048716
numero mas usado dia,0.044006


# Summit



In [193]:
to_pred = pd.read_csv("csv/trocafone_kaggle_test.csv")
features = pd.read_csv("features0,1.csv")

In [194]:
test = test.drop(["ultimo_mes","ultimo_dia","ultimo_hora","ultimo_dia_de_la_semana_activo"],axis = 1)

In [195]:
to_pred_completo = pd.merge(to_pred,test,on="person",how = "left").dropna()

In [196]:
to_pred_completo["label"] = grid_drop.best_estimator_.predict(to_pred_completo.drop(["person"],axis = 1))

In [197]:
to_pred = pd.merge(to_pred_completo[["person","label"]],to_pred,on="person",how="right")

In [198]:
to_pred = to_pred.fillna(to_pred["label"].median())

In [199]:
to_pred.to_csv("sumit0.0", index=False)

In [200]:
to_pred

,person,label
0,4886f805,0.011740
1,0297fc1e,0.064053
2,2d681dd8,0.013609
3,cccea85e,0.064655
4,4c8a8b93,0.068383
5,29ebb414,0.023057
6,3dc1950f,0.071115
7,8ea4c165,0.048834
8,d8cfe234,0.045309
9,d6bc64df,0.049415


5